In [1]:
# Import required packages
from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from shapely import geometry
from xcube_sh.sentinelhub import SentinelHub
import xarray as xr
import json
import IPython.display
import shapely.geometry

from sentinelhub import BBox, WmsRequest, DataSource, SHConfig
from functools import partial

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
# Set Sentinel Hub credentials
import os
sh_credentials = dict(client_id=os.environ['SH_CLIENT_ID'],
                      client_secret=os.environ['SH_CLIENT_SECRET']) # This is only provided when the Oauth credentials are created

# Sentinel-3 OLCI, Sentinel-3 SLSTR and Sentinel-5 layers are processed on different infrastructure, 
# which requires to used different end-point

sh_credentials.update(api_url='https://creodias.sentinel-hub.com')

In [3]:
f=open('countries.json')
data=json.load(f)
final=dict()
for i in range(0,len(data['features'])):
    #print(data['features'][i]['id'])
    final[data['features'][i]['id']]=data['features'][i]['geometry']['coordinates']

In [5]:
#EU_countries={'PER','VEN','CHL','ECU','GTM','BOL'}
EU_countries={'DEU'}

In [6]:
aoi = dict()
for country in EU_countries:
    if len(final[country])!=1:
        xs=0
        ys=0
        all_x=[]
        all_y=[]
        for i in range(0,len(final[country])):
            state=geometry.Polygon(final[country][i][0])
            geom = np.array(state.exterior.coords.xy)
            xs = geom[0]
            ys = geom[1]
            all_x.extend(xs)
            all_y.extend(ys)
    else:
        state=geometry.Polygon(final[country][0])
        geom = np.array(state.exterior.coords.xy)
        all_x = geom[0]
        all_y = geom[1]
    aoi[country]=(min(all_x),
                    min(all_y),
                    max(all_x),
                    max(all_y))

In [7]:
def caculate(geometry, timerange):
    
    cube_config = CubeConfig(dataset_name='S5PL2',
                         band_names=['O3','NO2','SO2','CO','CH4'],
                         tile_size=[512, 512],
                         bbox=geometry,
                         spatial_res=abs(geometry[2]-geometry[0])/512, # spatial resolution (approx. 20 m in degree)
                         time_range= timerange,
                         time_period='7D') 
    cube = open_cube(cube_config, **sh_credentials)
    return cube.mean(dim=["lon","lat"],skipna=True).to_dataframe()


In [8]:
import time 

start = time.time()
aoi_S5PL2 = list()

# Check length of provided aoi list OR dataframe
len_aoi = len(aoi)

# Define how many AOIs you want to process. For the demo we will use only one 
counter = 6

for i in aoi.keys():
    aoi_dict = dict()
    print("Processing: ", i)      
    timerange = ['2018-04-01', '2020-12-31']
    aoi_dict[str(i)] = caculate(aoi[str(i)], timerange)
    shape = aoi_dict[str(i)].shape
    nullCount = sum(aoi_dict[str(i)].isna().sum())
    print(f"Shape of 2018 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))

    aoi_S5PL2.append(aoi_dict)

end = time.time()

Processing:  DEU


/opt/conda/envs/eurodatacube-0.24.5/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Shape of 2018 DF: (288, 6), Count of Null values: 204


In [24]:
IPython.display.GeoJSON(shapely.geometry.box(*aoi["ITA"]).__geo_interface__)

<IPython.display.GeoJSON object>